<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/BERT_BunshoAC_IIC_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 乱数シードの固定

import os
import random
import numpy as np
import pandas as pd
import torch
import torchtext  # torchtextを使用

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [ ]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

In [ ]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.7)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already install

In [ ]:
# 教師
_train_data = pd.read_csv("https://stream.takky.org/rd/BERT_IIC/%E3%83%97%E3%83%AC%E3%83%A2%E3%83%AB%E3%82%A4%E3%83%A1%E3%83%BC%E3%82%B8.csv", encoding='utf-8', sep=',')
_train_data = _train_data.fillna({ _train_data.columns.values[2] : '空白'})
train_data = pd.concat([_train_data["検証データ"], _train_data["予測結果"]], axis=1)
train_data

,検証データ,予測結果
0,豪華で華やか,0
1,口当たりが滑らかで美味しい,0
2,なし,0
3,泡が旨い,0
4,矢沢永吉,0
...,...,...
5422,高品質,0
5423,美味しそうに飲んでいる,0
5424,きめが細かいクリーミーな泡,0
5425,飲みやすい,0


In [ ]:
train_data.isnull().any()

検証データ    False
予測結果     False
dtype: bool

In [ ]:
# 予測
_test_data = pd.read_csv("https://stream.takky.org/rd/BERT_IIC/%E3%83%97%E3%83%AC%E3%83%A2%E3%83%AB%E3%82%A4%E3%83%A1%E3%83%BC%E3%82%B8.csv", encoding='utf-8', sep=',')
_test_data = _test_data.fillna({ _test_data.columns.values[2] : '空白'})
test_data = pd.concat([_test_data["検証データ"], _test_data["予測結果"]], axis=1)
test_data

,検証データ,予測結果
0,豪華で華やか,華やか [C59]
1,口当たりが滑らかで美味しい,口当たり [C189]
2,なし,特になし [C28]
3,泡が旨い,泡 [C212]
4,矢沢永吉,矢沢永吉 [C11]
...,...,...
5422,高品質,高級感 [C29]
5423,美味しそうに飲んでいる,美味しい [C3]
5424,きめが細かいクリーミーな泡,泡・きめ細かい [C107]
5425,飲みやすい,飲みやすい [C116]


In [ ]:
categories = _train_data["予測結果"].unique()
print(categories)
print(len(categories))

['華やか [C59]' '口当たり [C189]' '特になし [C28]' '泡 [C212]' '矢沢永吉 [C11]'
 '高級 [C252]' '本格的 [C328]' '飲みごたえ・のみごたえ [C290]' '分からない [C126]'
 '飲みたくなる [C13]' '見る・飲む [C402]' '旨い [C247]' '泡・きめ細かい [C107]' '特になし [C43]'
 '女性向け' '特別 [C201]' 'さわやか [C34]' '爽やか [C106]' 'こだわって [C253]' 'おいしい [C8]'
 '香り・高い [C279]' '美味しい [C3]' '矢沢 [C291]' 'かわ良い・きれい [C48]' '贈り物 [C110]'
 '喉越し・のど越し [C45]' '神泡 [C21]' 'フルーティー・フルーティ [C124]' '泡' 'コク・おいしい [C142]'
 'プレミアム感・プレミアム [C256]' '売れている [C30]' '味・麦 [C2]' '洗練された' '素材・原料 [C237]'
 '買えない [C185]' '特になし [C41]' '香り [C20]' '高級ビール・高級そう [C392]' '上品 [C66]'
 '大人 [C373]' '美味しさ・本格的 [C82]' '特になし [C14]' '高い [C186]' '食事・合う [C36]'
 'TVCM [C271]' '特別感・本物感 [C12]' 'まろやか [C95]' '濃厚・旨味 [C233]' '高級感 [C4]'
 '自分・ご褒美 [C33]' '飲む [C103]' 'クリーミィ [C333]' '安心感 [C146]' '神泡サーバー・神泡 [C187]'
 '飲めない [C218]' '本物志向・個性的 [C192]' '高い・値段 [C42]' 'ご褒美 [C38]' '高級 [C302]'
 '美味しそう [C26]' 'サントリー' '品質 [C78]' '高級感 [C29]' '満足感・信頼感 [C18]'
 '香り・味 [C321]' 'リッチ・味 [C353]' '水にこだわり' '味・香り [C191]' 'グラス・飲む [C314]'
 '贅沢 [C102]' '石原さとみ [C23]' 'お

In [ ]:
train_df = pd.DataFrame({'text': train_data[train_data.columns.values[0]].values, 'label': train_data[train_data.columns.values[1]].values})
print(train_df.shape)

(5427, 2)


In [ ]:
train_df

,text,label
0,豪華で華やか,0
1,口当たりが滑らかで美味しい,0
2,なし,0
3,泡が旨い,0
4,矢沢永吉,0
...,...,...
5422,高品質,0
5423,美味しそうに飲んでいる,0
5424,きめが細かいクリーミーな泡,0
5425,飲みやすい,0


In [ ]:
test_df = pd.DataFrame({'text': test_data[test_data.columns.values[0]].values, 'label': test_data[test_data.columns.values[1]].values})
print(test_df.shape)

(5427, 2)


In [ ]:
test_df

,text,label
0,豪華で華やか,華やか [C59]
1,口当たりが滑らかで美味しい,口当たり [C189]
2,なし,特になし [C28]
3,泡が旨い,泡 [C212]
4,矢沢永吉,矢沢永吉 [C11]
...,...,...
5422,高品質,高級感 [C29]
5423,美味しそうに飲んでいる,美味しい [C3]
5424,きめが細かいクリーミーな泡,泡・きめ細かい [C107]
5425,飲みやすい,飲みやすい [C116]


In [ ]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

In [ ]:
# DataFrameにカテゴリーindexの列を作成
train_df["label_index"] = train_df["label"].map(dic_cat2id)
test_df["label_index"] = test_df["label"].map(dic_cat2id)

# label列を消去し、text, indexの順番にする
train_df = train_df.loc[:, ["text", "label_index"]]
test_df = test_df.loc[:, ["text", "label_index"]]

In [ ]:
# 順番をシャッフルする
train_df = train_df.sample(frac=1, random_state=123).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=123).reset_index(drop=True)

In [ ]:
# tsvファイルで保存する

# 訓練&検証データとする
train_df.to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
## あえて教師データを消してみる
train_df["label_index"] = [0] * len(train_df)

# テストデータとする
test_df.to_csv("./test.tsv", sep='\t', index=False, header=None)

In [ ]:
train_df

,text,label_index
0,美味い,0
1,滑らかな口当たり,0
2,サントリーホール,0
3,レジェンド,0
4,余韻を楽しめるイメージ,0
...,...,...
5422,わからない,0
5423,旨そう,0
5424,爽やか,0
5425,わからない,0


In [ ]:
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')

In [ ]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


In [ ]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

In [ ]:
dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

4070
1357
5427


In [ ]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}

In [ ]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
model.eval()
print('ネットワーク設定完了')

ネットワーク設定完了


In [ ]:
# BERTでベクトル化する関数を定義


def vectorize_with_bert(net, dataloader):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloader.batch_size

    # データローダーからミニバッチを取り出すループ
    for index, batch in enumerate(dataloader):
        # batchはTextとLableの辞書オブジェクト
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs = batch.Text[0].to(device)  # 文章
        labels = batch.Label.to(device)  # ラベル

        # 順伝搬（forward）計算
        with torch.set_grad_enabled(False):

            # Berに入力
            result = net(inputs)

            # sequence_outputの先頭の単語ベクトルを抜き出す
            vec_0 = result[0]  # 最初の0がsequence_outputを示す
            vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
            vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換

            # ベクトル化したデータをtorchリストにまとめる
            if index == 0:
                list_text = vec_0
                list_label = labels
            else:
                list_text = torch.cat([list_text, vec_0], dim=0)
                list_label = torch.cat([list_label, labels], dim=0)

    return list_text, list_label

In [ ]:
# DataLoaderをベクトル化版に変換
# 少し時間がかかります5分弱

list_text_train, list_label_train = vectorize_with_bert(model, dl_train)
list_text_eval, list_label_eval = vectorize_with_bert(model, dl_eval)
list_text_test, list_label_test = vectorize_with_bert(model, dl_test)

使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------


In [ ]:
# torchのリストをDatasetに変換

from torch.utils.data import TensorDataset

dataset_bert_train = TensorDataset(list_label_train.view(-1, 1), list_text_train)
dataset_bert_eval = TensorDataset(list_label_eval.view(-1, 1), list_text_eval)
dataset_bert_test = TensorDataset(list_label_test.view(-1, 1), list_text_test)

In [ ]:
# Dataloaderにする
from torch.utils.data import DataLoader

# 長すぎるので… batch_size = 1024
batch_size = 64

dl_bert_train = DataLoader(
    dataset_bert_train, batch_size=batch_size, shuffle=True, drop_last=True)
# drop_lastは最後のミニバッチがbatch_sizeに足りない場合は無視する

dl_bert_eval = DataLoader(
    dataset_bert_eval, batch_size=batch_size, shuffle=False)
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=batch_size, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

OVER_CLUSTRING_RATE = 10


class NetIIC(nn.Module):
    def __init__(self):
        super(NetIIC, self).__init__()

        # multi-headは今回しない
        self.conv1 = nn.Conv1d(1, 400, kernel_size=768, stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(400)
        self.conv2 = nn.Conv1d(1, 300, kernel_size=400, stride=1, padding=0)
        self.bn2 = nn.BatchNorm1d(300)
        self.conv3 = nn.Conv1d(1, 300, kernel_size=300, stride=1, padding=0)
        self.bn3 = nn.BatchNorm1d(300)

        self.fc1 = nn.Linear(300, 250)
        self.bnfc1 = nn.BatchNorm1d(250)

        # livedoorニュースの9カテゴリに対応するかな？と期待する9分類
        self.fc2 = nn.Linear(250, len(categories))

        # overclustering
        # 実際の想定よりも多めにクラスタリングさせることで、ネットワークで微細な変化を捉えられるようにする
        #self.fc2_overclustering = nn.Linear(250, 9*OVER_CLUSTRING_RATE)
        self.fc2_overclustering = nn.Linear(250, len(categories)*OVER_CLUSTRING_RATE)

    def forward(self, x):
        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn1(self.conv1(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn2(self.conv2(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn3(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x_prefinal = F.relu(self.bnfc1(self.fc1(x)))

        # multi-headは使わず
        y = F.softmax(self.fc2(x_prefinal), dim=1)
        y_overclustering = F.softmax(self.fc2_overclustering(
            x_prefinal), dim=1)  # overclustering

        return y, y_overclustering

In [ ]:
import torch.nn.init as init


def weight_init(m):
    """重み初期化"""
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        # Xavier
        # init.xavier_normal_(m.weight.data)

        # He
        init.kaiming_normal_(m.weight.data)

        if m.bias is not None:
            init.normal_(m.bias.data)

In [ ]:
# IISによる損失関数の定義
# 参考：https://github.com/RuABraun/phone-clustering/blob/master/mnist_basic.py
import sys


def compute_joint(x_out, x_tf_out):
    bn, k = x_out.size()
    assert (x_tf_out.size(0) == bn and x_tf_out.size(1) == k), '{} {} {} {}'.format(
        bn, k, x_tf_out.size(0), x_tf_out.size(1))

    p_i_j = x_out.unsqueeze(2) * x_tf_out.unsqueeze(1)  # bn, k, k
    p_i_j = p_i_j.sum(dim=0)  # k, k
    p_i_j = (p_i_j + p_i_j.t()) / 2.  # symmetrise
    p_i_j = p_i_j / p_i_j.sum()  # normalise
    return p_i_j


def IID_loss(x_out, x_tf_out, EPS=sys.float_info.epsilon):
    # has had softmax applied
    bs, k = x_out.size()
    p_i_j = compute_joint(x_out, x_tf_out)
    assert (p_i_j.size() == (k, k))

    p_i = p_i_j.sum(dim=1).view(k, 1).expand(k, k)
    p_j = p_i_j.sum(dim=0).view(1, k).expand(k, k)

    # avoid NaN losses. Effect will get cancelled out by p_i_j tiny anyway
    # これはPyTorchのバージョン1.3以上だとエラーになる
    # https://discuss.pytorch.org/t/pytorch-1-3-showing-an-error-perhaps-for-loss-computed-from-paired-outputs/68790/3
    #p_i_j[(p_i_j < EPS).data] = EPS
    #p_j[(p_j < EPS).data] = EPS
    #p_i[(p_i < EPS).data] = EPS

    p_i_j = torch.where(p_i_j < EPS, torch.tensor(
        [EPS], device=p_i_j.device), p_i_j)
    p_j = torch.where(p_j < EPS, torch.tensor([EPS], device=p_j.device), p_j)
    p_i = torch.where(p_i < EPS, torch.tensor([EPS], device=p_i.device), p_i)

    # https://qiita.com/Amanokawa/items/0aa24bc396dd88fb7d2a
    # 参考に、重みalphaを追加

    alpha = 2.0
    loss = (- p_i_j * (torch.log(p_i_j) - alpha *
                       torch.log(p_j) - alpha*torch.log(p_i))).sum()

    return loss

In [ ]:
# データにノイズを加える関数の定義
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor_std = list_text_train.std(dim=0).to(device)


def perturb_data(x):
    y = x.clone()
    noise = torch.randn(len(tensor_std)).to(device)*tensor_std*2.0
    noise = noise.expand(x.shape[0], -1)
    y += noise

    return y

In [ ]:
# 学習関数の定義

def train(total_epoch, model, train_loader, optimizer, device):

    # ネットワークを訓練モードに
    model.train()

    # 学習率のスケジューラーCosAnnealing
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=2, T_mult=2, eta_min=0)

    for epoch in range(total_epoch):
        for batch_idx, (target, data) in enumerate(train_loader):

            # 学習率変化
            scheduler.step()
            
            data_perturb = perturb_data(data)  # ノイズを与え、変換したデータを作る

            # GPUに送れる場合は送る
            data = data.to(device)
            data_perturb = data_perturb.to(device)

            # 最適化関数の初期化
            optimizer.zero_grad()

            # ニューラルネットワークへ入れる
            output, output_overclustering = model(data)
            output_perturb, output_perturb_overclustering = model(data_perturb)

            # 損失の計算
            loss1 = IID_loss(output, output_perturb)
            loss2 = IID_loss(output_overclustering, output_perturb_overclustering)
            loss = loss1 + loss2

            # 損失を減らすように更新
            loss.backward()
            optimizer.step()

        # ログ出力
        if epoch % 50 == 0:
            print('Train Epoch {} \tLoss1: {:.6f} \tLoss2: {:.6f} \tLoss_total: {:.6f}'.format(
                epoch, loss1.item(), loss2.item(), loss1.item()+loss2.item()))

    return model, optimizer

In [ ]:
# 学習の実施(5分弱)

# モデルの用意
net = NetIIC()
net.apply(weight_init)
net.to(device)

# 最適化関数
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4) 

total_epoch = 1000

model_trained, optimizer = train(
    total_epoch, net, dl_bert_train, optimizer, device)

Train Epoch 0 	Loss1: -9.181230 	Loss2: -13.619122 	Loss_total: -22.800351
Train Epoch 50 	Loss1: -11.349316 	Loss2: -15.857475 	Loss_total: -27.206791
Train Epoch 100 	Loss1: -11.702747 	Loss2: -16.347536 	Loss_total: -28.050283
Train Epoch 150 	Loss1: -11.670151 	Loss2: -16.256321 	Loss_total: -27.926472
Train Epoch 200 	Loss1: -11.526920 	Loss2: -16.251175 	Loss_total: -27.778095
Train Epoch 250 	Loss1: -11.821992 	Loss2: -16.399376 	Loss_total: -28.221368
Train Epoch 300 	Loss1: -11.834357 	Loss2: -16.533428 	Loss_total: -28.367785
Train Epoch 350 	Loss1: -11.954149 	Loss2: -16.491121 	Loss_total: -28.445271
Train Epoch 400 	Loss1: -11.936567 	Loss2: -16.502804 	Loss_total: -28.439371
Train Epoch 450 	Loss1: -11.802656 	Loss2: -16.447466 	Loss_total: -28.250122
Train Epoch 500 	Loss1: -11.758309 	Loss2: -16.344278 	Loss_total: -28.102588
Train Epoch 550 	Loss1: -11.843164 	Loss2: -16.412766 	Loss_total: -28.255930
Train Epoch 600 	Loss1: -11.761138 	Loss2: -16.547283 	Loss_total: -

In [ ]:
# モデル分類のクラスターの結果を確認する
import numpy as np

# ミニバッチサイズ1のテスト用のDataLoaderを用意
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=1, shuffle=False)


def test(model, device, test_loader):
    model.eval()

    out_targs = []
    ref_targs = []

    # 出力用のリストを用意
    total_num = len(test_loader)
    # index, (target_label, inferenced_label)
    output_list = np.zeros((total_num, 2))

    with torch.no_grad():
        for batch_idx, (target, data) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            outputs, outputs_overclustering = model(data)

            # 分類結果をリストに追加
            out_targs.append(outputs.argmax(dim=1).cpu())
            ref_targs.append(target[0].cpu())

            # 結果をリストにまとめる
            output_list[batch_idx, 0] = target[0][0].cpu()  # 正解ラベル
            output_list[batch_idx, 1] = outputs.argmax(dim=1).cpu()  # 予測ラベル

    out_targs = torch.cat(out_targs)
    ref_targs = torch.cat(ref_targs)

    return out_targs.view(-1, 1).numpy(), ref_targs.numpy(), output_list

In [ ]:
# テストデータで推論を実施
out_targs, ref_targs, output_list = test(model_trained, device, dl_bert_test)
print(len(dl_bert_test))

5427


In [ ]:
# 混同行列（的な）を作る
#matrix = np.zeros((9, 9))
matrix = np.zeros((len(categories), len(categories)))

# 縦にlivedoorニュースの正解クラスを、横に判定されたクラスの頻度表を作成
for i in range(len(out_targs)):
    row = ref_targs[i]
    col = out_targs[i]
    matrix[row][col] += 1

np.set_printoptions(suppress=True)
print(matrix)

[[1. 1. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# クラスタの結果を確認
#「sports-watch」の5番目のクラスタの文章、7番目のクラスタの文章
#「topic-news」の5番目のクラスタの文章、7番目のクラスタの文章
# を確認して、クラスタ5とクラスタ7の特徴を見てみます。

import pandas as pd

df2 = pd.DataFrame(output_list)
df2.columns=["正解クラスNo", "推定クラスタNo"]
df2["正解クラス"] = df2["正解クラスNo"].map(dic_id2cat)
df2 = df2[["正解クラス", "正解クラスNo", "推定クラスタNo"]]

In [ ]:
df_test_txt = pd.read_csv('test.tsv', encoding='utf-8', sep='\t', header=None)
df_test_txt.columns=["TEXT", "CLASS"]

In [ ]:
result_df = pd.concat([df2, df_test_txt["TEXT"]], axis=1)
result_df.to_csv("./BERT_BunshoAC_IIC_3.tsv", encoding='utf-8', sep='\t')

In [ ]:
from google.colab import files
files.download("./BERT_BunshoAC_IIC_3.tsv")

In [ ]:
result_df[result_df["推定クラスタNo"] == 1.]

,正解クラス,正解クラスNo,推定クラスタNo,TEXT
65,飲む [C103],51.0,1.0,ゆっくりと楽しめる
112,贈り物 [C110],24.0,1.0,贈り物に合う
164,まろやか [C95],47.0,1.0,まろやか
232,まろやか [C95],47.0,1.0,まろやか
252,特になし [C41],36.0,1.0,くせがない
...,...,...,...,...
5311,フルーティー・フルーティ [C124],27.0,1.0,イカシテる
5321,飲む [C103],51.0,1.0,ゆったり飲む
5322,矢沢 [C291],22.0,1.0,吉野家
5334,神泡サーバー・神泡 [C187],54.0,1.0,神泡サーバー


In [ ]:
result_df[result_df["推定クラスタNo"] == 92.]

,正解クラス,正解クラスNo,推定クラスタNo,TEXT
267,自分・ご褒美 [C33],50.0,92.0,自分へのご褒美
430,サントリー,61.0,92.0,サントリーの水育の活動
715,香り・高い [C279],20.0,92.0,最高のビール
1103,本物志向・個性的 [C192],56.0,92.0,個性のある好みの分かれる味
1178,渋い [C85],86.0,92.0,ホップの苦味
1322,自分・ご褒美 [C33],50.0,92.0,自分へのご褒美
1348,自分・ご褒美 [C33],50.0,92.0,自分へのご褒美
1400,自分・ご褒美 [C33],50.0,92.0,明日への活力
1486,高級感・プレミアム感 [C330],88.0,92.0,息子の子供時代
1562,本格的 [C328],6.0,92.0,本物、本格的


In [ ]:
print("推定クラスタNo : カウント")
for idx, cnt in result_df["推定クラスタNo"].value_counts().sort_index().items():
  print(str(idx) + "     :     " + str(cnt))

推定クラスタNo : カウント
0.0     :     103
1.0     :     95
3.0     :     4
5.0     :     28
6.0     :     90
8.0     :     12
10.0     :     17
11.0     :     27
12.0     :     36
13.0     :     109
14.0     :     268
15.0     :     153
16.0     :     40
17.0     :     1
18.0     :     188
20.0     :     134
22.0     :     86
23.0     :     9
24.0     :     63
27.0     :     19
28.0     :     130
29.0     :     94
32.0     :     15
34.0     :     27
35.0     :     8
36.0     :     69
37.0     :     208
38.0     :     45
39.0     :     9
40.0     :     107
43.0     :     61
45.0     :     2
46.0     :     4
47.0     :     115
48.0     :     55
49.0     :     67
52.0     :     950
53.0     :     113
54.0     :     48
55.0     :     8
56.0     :     63
57.0     :     38
58.0     :     6
59.0     :     18
60.0     :     28
62.0     :     90
65.0     :     28
66.0     :     163
67.0     :     53
68.0     :     41
69.0     :     1
70.0     :     70
71.0     :     26
72.0     :     74
73.0     :     